In [2]:
import sys
sys.path.append('../../src')

import pandas as pd
import re 
from data_manager import data_helper

In [3]:
original_data = pd.read_excel(data_helper["input"] / "main_dataset.xlsx")
print(len(original_data))

130303


In [4]:
original_data.head()

,Код,Наименование,НаименованиеПолное,Группа,ВидНоменклатуры,ЕдиницаИзмерения
0,НИ-IS0032430,Cветильник LED панель 50W 6500k IP40 1200мм ДП...,Cветильник LED панель 50W 6500k IP40 1200мм ДП...,АВАНСОВЫЙ ОТЧЕТ,Инвентарь и хозяйственные принадлежности,шт
1,НИ-IS0040225,"Cветильник потолочный,Osairous,Белый, 30Вт LED...","Cветильник потолочный,Osairous,Белый, 30Вт LED...",АВАНСОВЫЙ ОТЧЕТ,Сырье и материалы,шт
2,НИ-IS0014483,Cистема IP-DECT Yealink W80DM контроллер микро...,Cистема IP-DECT Yealink W80DM контроллер микро...,АВАНСОВЫЙ ОТЧЕТ,Инвентарь и хозяйственные принадлежности,шт
3,НИ-IS0047648,Cпрей для ванной комнаты Sanfor (Санфор) 500мл,Cпрей для ванной комнаты Sanfor (Санфор) 500мл,АВАНСОВЫЙ ОТЧЕТ,Сырье и материалы,шт
4,НИ-IS0047649,Cпрей ультрабелый Sanfor (Санфор) 500мл,Cпрей ультрабелый Sanfor (Санфор) 500мл,АВАНСОВЫЙ ОТЧЕТ,Сырье и материалы,шт


In [5]:
dop_data = pd.read_excel(data_helper["input"] / "dop_data.xlsx")
print(len(dop_data))

7185


In [6]:
dop_data.head()

,Склад,Материал,Наименование материала,БЕИ,место хранения,весовые характеристики
0,R010,20140460.0,Бейсболка лет М027-03 р.61,ШТ,ТС,0.0001
1,R010,20438940.0,Бейсболка_РН_ХБ_То_54,ШТ,ТС,0.0001
2,R010,20589892.0,Бейсболка_РН_ХБТо_55-57,ШТ,ТС,0.0001
3,R010,20589902.0,Бейсболка_РН_ХБТо_58-61,ШТ,ТС,0.0001
4,R010,20569863.0,"Белье 2/08/797 2/01/797, 128-132/182-188",К-Т,ТС,0.0006


In [7]:
names = {}
names_beffur = {}
for i, row in original_data.iterrows():
    flag = False
    names.setdefault(row["ВидНоменклатуры"], set())
    for k, n in names.items():
        if k == row["ВидНоменклатуры"]:
            continue
        
        if row["Наименование"] in n:
            flag = k
            break

    if flag:
        # print(row["ВидНоменклатуры"], row["Наименование"], flag)
        names_beffur.setdefault(flag, set())
        names_beffur[flag].add(row["Наименование"])
        continue
    
    names[row["ВидНоменклатуры"]].add(row["Наименование"])

In [8]:
for type, t_names in sorted(names.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"{type}-{len(t_names)}")

Запасные части-49119
Сырье и материалы-23297
Инвентарь и хозяйственные принадлежности-20179
Спецодежда-10664
Спецоснастка-2631
Строительные материалы-2525
Аварийный инструмент и инвентарь-2144
Запасные части к БО-2112
Основное средство-2042
Услуга-2039
Оснастка-1575
Инструмент-1217
Оргтехника и программное обеспечение-1195
Оборудование малоценное-1132
Химический реагент-1097
Масла и смазки-1078
Товары-1008
Запасные части к ТС-989
Прочие материалы-788
Труба обсадная-455
Нематериальный актив-446
Труба бурильная-263
Панели ситовые-258
Тара и тарные материалы-157
Техника безопасности и охрана труда-138
Запасные части к ПВО-129
Цемент-129
Топливо-48
Лом металлический-22
Труба общего назначения-20
Оборудование (объекты основных средств) сч. 08.4-12
Оборудование к установке-7
Отходы бурения-7
Транспортные средства-5
Раствор буровой-4
Материалы-3
Корзины ситовые-1
Специальная оснастка и специальная одежда на склад-1
Запчасти-1


In [9]:
CLASSES = {
    "Материалы": 1,
    "Оборудование и техника": 2,
    "Запасные части": 3,
    "Инструменты и оснастка": 4,
    "Прочие материальные ценности": 5,
    "IT и ПО": 6,
    "Одежда и СИЗ": 7,
    "Нематериальные активы": 8,
    "Услуги и отходы": 9,
    "Прочее": 10
}

NIP_CLASS = {
    1: ["Сырье и материалы", "Строительные материалы", "Цемент", "Масла и смазки", "Химический реагент", "Материалы", "Топливо", "Раствор буровой"],
    2: ["Оборудование малоценное", "Оборудование к установке", "Оборудование (объекты основных средств) сч. 08.4", "Основное средство", "Транспортные средства", 
        "Труба обсадная", "Труба общего назначения", "Труба бурильная"],
    3: ["Запасные части", "Запчасти", "Запасные части к ТС", "Запасные части к БО", "Запасные части к ПВО"],
    4: ["Инструмент", "Оснастка", "Спецоснастка", "Специальная оснастка и одежда"],
    5: ["Инвентарь и хозяйственные принадлежности", "Аварийный инструмент и инвентарь", "Техника безопасности и охрана труда", "Тара и тарные материалы", "Лом металлический"],
    6: ["Оргтехника и программное обеспечение"],
    7: ["Спецодежда", "Техника безопасности", "Специальная оснастка и специальная одежда на склад"],
    8: ["Нематериальный актив"],
    9: ["Услуга", "Отходы бурения"],
    10: ["Товары", "Панели ситовые", "Корзины ситовые", "Прочие материалы", "Трубы (обсадные, бурильные, общие)", "Цемент"]
}

In [10]:
data = []
for category, items in names.items():
    for class_id, val in NIP_CLASS.items():
        if category in val:
            for name in items:
                data.append((name, class_id))
            break
    else:
        print(category)

In [11]:
len(data)

128937

In [12]:
from sklearn.model_selection import train_test_split

texts, labels = zip(*data)
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1,2))),
    ("clf", LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))

Accuracy: 0.8674965100046533


In [14]:
result = model.predict(["Шуруповерт Bosch 12V"])

In [15]:
result

array([5])

In [ ]:
tokens = tokenizer("Шуруповерт Bosch 12V", return_tensors="pt")
output = model(**tokens)
predicted_class_id = torch.argmax(output.logits, dim=1).item()
label = le.inverse_transform([predicted_class_id])[0]

In [ ]:
import transformers
print(transformers.__version__)

4.55.2
